# Clone my repo to get all paths

In [ ]:
# So Colab can use the python files...
!git clone --single-branch -b develop https://github.com/muellevin/Studienarbeit.git
%cd Studienarbeit/Detection_training

 and set all up

In [ ]:
import os
import Tensorflow.scripts.Paths as pp

CUSTOM_MODEL_NAME = 'raccoonModel_50ep_b16_efficientdetlit0_17070' 

paths = pp.paths
paths.setup_paths()

OUTPUT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'export')
SAVED_MODEL = os.path.join(OUTPUT_PATH, 'saved_model')
TFLITE_EXPORT = os.path.join(OUTPUT_PATH, 'tfliteexport')
os.makedirs(OUTPUT_PATH, exist_ok=True)


# Setup for EfficientDet-Lite

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# Upload images

In [ ]:
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/Studienarbeit'

if os.name =='posix':
    from google.colab import drive

    drive.mount('/content/drive', force_remount=True)

import shutil
shutil.copy(os.path.join(GOOGLE_DRIVE_PATH, pp.DATASET_NAME), paths.IMAGE_PATH)

if os.path.exists(pp.DATASET):
  !tar -zxvf {pp.DATASET}

To lazy to do the  conversion again locally

In [ ]:
from Tensorflow.scripts.csv_to_voc import convert_csv_to_voc


convert_csv_to_voc(os.path.join(paths.TRAINSET_PATH, pp.CSV_FILE_NAME), paths.TRAINSET_PATH)
convert_csv_to_voc(os.path.join(paths.TESTSET_PATH, pp.CSV_FILE_NAME), paths.TESTSET_PATH)
convert_csv_to_voc(os.path.join(paths.DEVSET_PATH, pp.CSV_FILE_NAME), paths.DEVSET_PATH)

In [ ]:
label_map = {} # dictionary to store class names
for label in pp.LABELS:
    label_map[label['id']] = label['name']
# prepare data for training
train_data = object_detector.DataLoader.from_pascal_voc(
        paths.TRAINSET_PATH, paths.TRAINSET_PATH, label_map=label_map)
validation_data = object_detector.DataLoader.from_pascal_voc(
        paths.TESTSET_PATH, paths.TESTSET_PATH, label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(
        paths.DEVSET_PATH, paths.DEVSET_PATH, label_map=label_map)

In [ ]:
# choose model
spec = object_detector.EfficientDetLite0Spec()

In [ ]:
# train model
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=34, 
                               batch_size=16, 
                               train_whole_model=True)

In [ ]:
# evaluate
model.evaluate(test_data)

In [ ]:
# convert to tflite
model.export(export_dir=TFLITE_EXPORT, tflite_filename=CUSTOM_MODEL_NAME + '.tflite', label_filename='labels.txt',
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

# and evaluate this
model.evaluate_tflite(os.path.join(TFLITE_EXPORT, CUSTOM_MODEL_NAME + '.tflite'), test_data)

# Compile for TPU

In [ ]:
# Compile for TPU
# ! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

# ! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

# ! sudo apt-get update

# ! sudo apt-get install edgetpu-compiler

NUMBER_OF_TPUS =  1

!edgetpu_compiler {os.path.join(TFLITE_EXPORT, CUSTOM_MODEL_NAME + '.tflite')} -d --num_segments=$NUMBER_OF_TPUS

In [ ]:
# zip and download
!zip -r {os.path.join(GOOGLE_DRIVE_PATH, CUSTOM_MODEL_NAME + '.zip')} {TFLITE_EXPORT}